# Workshop Time Series

In this workshop you will practice performing time series analysis.

Lesson goal: The student applies time series forecasting techniques to make predictions using a time series data set. This lesson goal relates to final qualification "data analytics".


We will be using a dataset containing energy consumption data for a heat pump. It contains the following columns:

- timestamp: timestamp
- E_apparaten: energy use of other devices in kWh
- E_warmtepomp: energy use of heatpump in kWh
- E_verbruik: total energy use in kWh
- Overschot: energy returned to grid
- hp_AAN: heatpump on or off
- diff_temp_woonk: temperature living room
- diff_temp_slaapk: temperature bedroom
- Tussendeur: inside door open / closed
- Buitendeur: outside door open / closed
- Raam: window open / closed
- Dag: weekday
- day_of_week: weekday (same as "Dag")
- month: month
- year: year

In this workshop we will focus on the E_warmtepomp column

In [ ]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from statsmodels.graphics import tsaplots
from statsmodels.tsa.stattools import acf
import statsmodels.tsa.api as tsa
from scipy.signal import savgol_filter, butter, filtfilt
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error


In [ ]:
# Read in the data
dfhp = pd.read_csv("data/heatpump_clean_filtered.csv")
dfhp['timestamp_dt'] = pd.to_datetime(dfhp['timestamp']) # Create a copy of the timestamp that is converted to a proper date time stamp object so Python can analyze it better.
dfhp

## Initial analysis

As there are 100,000 rows for our initial experiments for our first analyses we will summarize them or zoom on sub selections to avoid having to spend too much time waiting for calculations to complete. Later on you will work on the full data set.

For starters, lets sum the E_warmptepomp data for each day. Then plot the contents to see what we're dealing with.

In [ ]:
# To be able to sum by day we need to create a column that contains the day only.
# Note we use the to_datetime function to convert the date string to something
# that Python recognizes as a date object.
dfhp['datum'] = pd.to_datetime(dfhp['timestamp'].replace(' .*$', '', regex=True))
# Now sum the E_warmtepomp values for each day. Reset the index to obtain a dataframe instead of a groupby object.
df_sum = dfhp[['datum', 'E_warmtepomp']].groupby(by='datum').sum().reset_index()

plt.plot(df_sum['datum'], df_sum['E_warmtepomp'])
plt.title('Heat pump energy use / day')
plt.ylabel('Heat pump energy use (kWh)')
plt.xlabel('Date')
plt.show()



Clearly there are some interesting things going on with the data. For our first experiments, these are inconvenient, however, so let's first zoom in on a sub selection of data, say from 2020-01-01 to 2020-04-01

In [ ]:
df_sum_sel = df_sum[(df_sum['datum'] >= '2020-01-01') & (df_sum['datum'] < '2020-04-01')].reset_index(drop=True) # reset index is needed for ARIMA, later on.
plt.plot(df_sum_sel['datum'], df_sum_sel['E_warmtepomp'])
plt.title('Heat pump energy use / day between 2020-01-01 and 2020-04-01')
plt.ylabel('Heat pump energy use (kWh)')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.show()


There does not seem to be an obvious pattern here. Let's create an autocorrelation function plot to see if this intuition is correct.

In [ ]:
acf_plot = tsaplots.plot_acf(df_sum_sel["E_warmtepomp"], lags=30, title='Autocorrelation plot (heat pump energy use / day between 2020-01-01 and 2020-04-01)')

# Adjust the plot size
acf_plot.set_figheight(5)
acf_plot.set_figwidth(10)

# Show the plot
plt.show()

Autocorrelation is extremely weak, confirming our intuition that there is no obvious repeating pattern in the data.

However, there may be a seasonal effect in the data, albeit a weak one. Can you spot it?

Let's see if decomposition makes the effect more obvious.

In [ ]:

# Decompose the time series
result = seasonal_decompose(df_sum_sel["E_warmtepomp"], model="additive", period=30)

# Plot the decomposed components
decomposed = result.plot()

decomposed.set_figheight(8)
decomposed.set_figwidth(10)
decomposed.suptitle("Time Series Decomposition (heat pump energy use / day between 2020-01-01 and 2020-04-01)")

plt.show()

There seems to be a clear, albeit small, trend in the data. Perhaps filtering would make this effect more obvious?

In [ ]:
cutoff_frequency = 0.1  # Cutoff frequency as a fraction of the sampling rate
filter_order = 3  # Filter order
b, a = butter(filter_order, cutoff_frequency, btype="low", analog=False)
butterworth_filtered = filtfilt(b, a, df_sum_sel['E_warmtepomp'])

plt.plot(df_sum_sel['datum'], butterworth_filtered)
plt.title('Heat pump energy use / day between 2020-01-01 and 2020-04-01 (filtered)')
plt.ylabel('Heat pump energy use (kWh)')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.show()

Play around with the cutoff_frequency parameter to see if you can make the effect more pronounced.

## Try it yourself: analyze the full data set

Now try it yourself. Using the complete data set, analyse it for seasonality and trends. Make sure to use the dfhp['timestamp_dt'] column for the timestamp.

Play around with the values for the lags parameter and the period parameter

In [ ]:
# Your code for the ACF plot goes here. Try changing the lags parameter.



In [ ]:
# Your code for the decomposition goes here. Be sure to try changing the period parameter.
# If you have time, try filtering the data before decomposing it.




## Differencing

Our data has a (weak) trend. We can get rid of this trend by differencing. Doing so manually would be possible (refer to the lecture demos for an example) but Pandas has a differencing function built in so let's use that.

For the example we will again use the summarized data set.

In [ ]:
# Without a value for the periods parameter it defaults to 1
sum_diff = df_sum_sel['E_warmtepomp'].diff(periods=1)
plt.plot(df_sum_sel['datum'], sum_diff)
plt.title('Differenced heat pump energy use / day between 2020-01-01 and 2020-04-01')
plt.ylabel('Difference')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.show()

Whereas before we had a slight downward trend after differencing that trend is gone.

To verify that the autocorrelation is now (mostly) gone, we can make an autocorrelation function plot for the differenced time series.



In [ ]:
# We need to dropna as the first value is now empty (nothing to diff with!)
acf_plot = tsaplots.plot_acf(df_sum_sel["E_warmtepomp"].diff().dropna(), lags=30, title='Autocorrelation plot (differenced heat pump energy use / day between 2020-01-01 and 2020-04-01)')

# Adjust the plot size
acf_plot.set_figheight(5)
acf_plot.set_figwidth(10)

# Show the plot
plt.show()

Correlations, especially the second one, are now much weaker. The trend may be too, but this is difficult to see as it was quite weak to begin with.

Let's try to decompose the differenced data.

In [ ]:
# Decompose the time series
result = seasonal_decompose(df_sum_sel["E_warmtepomp"].diff().dropna(), model="additive", period=30)

# Plot the decomposed components
decomposed = result.plot()

decomposed.set_figheight(8)
decomposed.set_figwidth(10)
decomposed.suptitle("Time Series Decomposition (differenced)")

plt.show()

The trend is clearly gone.

## Try it yourself: differencing

Try differencing the entire data set. Does this remove the trend? And does this remove the seasonality, too?

In [ ]:
# Your code goes here.



## Forecasting

Now that we have analyzed our time series we can attempt to make forecasts.

We will use the ARIMA model from the statsmodels.tsa.api package. Perhaps confusingly this is also the model to use for ARMA models - by setting the differencing order to 0 (which of course makes sense).



In [ ]:
# Parameters for the ARIMA model
num_previous_terms = 1 # Commonly called p
differencing_order = 0 # Commonly called d. Should be 0 for ARMA
num_terms_moving_average = 1 # Commonly called q

model = tsa.ARIMA(df_sum_sel['E_warmtepomp'], order=(num_previous_terms, differencing_order, num_terms_moving_average))

fitted_model = model.fit()

fitted_model.summary()


Now that we have a model, we can make predictions.

In [ ]:
num_values_to_forecast = 10 # defaults to 1
print(fitted_model.forecast(num_values_to_forecast))

Let's eyeball the quality of our forecasts by plotting them.

In [ ]:
data_inc_forecasts = pd.concat([df_sum_sel['E_warmtepomp'], fitted_model.forecast(num_values_to_forecast)])
data_inc_forecasts.plot()
plt.title('Heat pump energy use inc prediction (ARMA)')
plt.show()

The forecasts are quite bad - a nearly straight line. 

In [ ]:
# Play around with these parameters until you get a reasonable looking result.
num_previous_terms = 4 # Commonly called p
differencing_order = 3 # Now set it to 1 - or even higher to get rid of higher order trends and seasonality
num_terms_moving_average = 2 # Commonly called q

model_arima = tsa.ARIMA(df_sum_sel['E_warmtepomp'], order=(num_previous_terms, differencing_order, num_terms_moving_average), enforce_invertibility=False)

fitted_model_arima = model_arima.fit()

data_inc_forecasts = pd.concat([df_sum_sel['E_warmtepomp'], fitted_model_arima.forecast(num_values_to_forecast)])
data_inc_forecasts.plot()
plt.title('Heat pump energy use inc prediction (ARIMA)')
plt.show()


Still not very good. Our data is simply too noisy to work well. How bad is it? To find this out, we need to split the data into a train set and a test set. FOR THIS WE DO NOT USE scikit-learn's train_test_split!

Then we make predictions on the train set and compare those to the test set using mean_absolute_error and mean_absolute_percentage_error from sklearn.metrics.


In [ ]:
# Create a train set and a test set. Last 10 readings are the test set.
train = df_sum_sel['E_warmtepomp'][:-10]
test = df_sum_sel['E_warmtepomp'][-10:]

# Play around with these parameters until you get a reasonable result.
num_previous_terms = 1 # Commonly called p
differencing_order = 2 # Now set it to 1 - or even higher to get rid of higher order trends and seasonality
num_terms_moving_average = 1 # Commonly called q

# Train a model
model_arima = tsa.ARIMA(train, order=(num_previous_terms, differencing_order, num_terms_moving_average), enforce_invertibility=False)

fitted_model_arima = model_arima.fit()

forecasts = fitted_model_arima.forecast(test.size)

# Calculate the MAPE
print(f'The MAPE is {mean_absolute_percentage_error(test, forecasts) * 100:.0f}%')


Now try it yourself on the complete data set. Create a train set and a test set (manually!)

For the parameters: play around with the number of previous terms and previous error terms to use. Note that larger values will lead to longer run times.

Also, try different test set sizes. What does this tell you about the predictive power of this model?

Finally, you may obtain better results if you focus on part of the data only, either the range before april or the range after (as there is a clear cutoff point in the first week of april)

In [ ]:
# Your code goes here


It is likely your results on the full data set, with 4 readings per hour, are far worse than the results for the summarized data set, with only one summed value per day, if you predict more than a handful of values.

If you think about it, the reasons for this are obvious - but it is nonetheless disappointing.

While time series forecasting can be a powerful technique, it only works on data sets where there is a meaningful relationship between data points. Often you will need to do additional work before such a relationship is produced, as we did in the examples above where we summed the individual readings per day.